In [227]:
import pandas as pd
from arch import arch_model
import jqdatasdk as jq
import numpy as np
import time,datetime
import os
import statsmodels.api as sm
from arch import arch_model

In [228]:
jq.auth('13918852005','960312Lsc')
# jq.auth('15821912507','912507')

In [242]:
# 引入日历效应哑变量 - 星期
sample_df = pd.read_csv('E:/Stock_Data/stock_return_data/001896.XSHE.csv',index_col=0)
trade_day_serises = list(sample_df.index)

trade_day_df = pd.DataFrame(trade_day_serises)
trade_day_df.columns = ['weekday']
trade_day_df['weekday'] = trade_day_df['weekday'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d").weekday()+1)

dummy_df = pd.get_dummies(trade_day_df['weekday'])
dummy_df = dummy_df[[1,2,3,4,5]]
dummy_df.index = sample_df.index

In [95]:
# 引入日历效应变量 - 假日
sample_df = pd.read_csv('E:/Stock_Data/stock_return_data/001896.XSHE.csv',index_col=0)
trade_day_serises = list(sample_df.index)

trade_day_df = pd.DataFrame(trade_day_serises)
trade_day_df.columns = ['holiday']
trade_day_df['holiday'] = trade_day_df['holiday'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))
trade_day_df['holiday_1'] = trade_day_df['holiday'].shift(1)

trade_day_df['holiday'] = trade_day_df['holiday'] - trade_day_df['holiday_1']
trade_day_df = trade_day_df.fillna(1)
trade_day_df['holiday'] = trade_day_df['holiday'].apply(lambda x : int(str(x)[:2])-1)
trade_day_df.iloc[0,0] = 0

holiday_df = trade_day_df[['holiday']]
holiday_df.index = sample_df.index

In [274]:
# 指数角度检验 - 全时段

index_save_dct = {1:0,2:0,3:0,4:0,5:0}
index_p_save_dct = {1:0,2:0,3:0,4:0,5:0}
for index_file in ['SZCZ_lnr_rv_w_m_ntd_080101-','SZZS_lnr_rv_w_m_ntd_080101-','CYBZ_lnr_rv_w_m_ntd_100601-']:
    index_df = pd.read_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/'+index_file+'190131adj.csv',index_col=0)
# index_df = pd.read_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZCZ_lnr_rv_w_m_ntd_080101-190131adj.csv',index_col=0)
# index_df = pd.read_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZZS_lnr_rv_w_m_ntd_080101-190131adj.csv',index_col=0)
# index_df = pd.read_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/CYBZ_lnr_rv_w_m_ntd_100601-190131adj.csv',index_col=0)

    index_df = index_df[index_df.index>'2011-01-04']
    index_df = index_df[index_df.index<'2018-12-28']
    X = dummy_df.iloc[:1943,:]
    y = index_df['lnR']

    # 用OLS
#     X = sm.add_constant(X)
    est = sm.OLS(y,X)
    result = est.fit()
    for i in range(5):
        index_save_dct[i+1] = '%.6f' %list(result.params)[i]
    for i in range(5):
        index_p_save_dct[i+1] = '%.6f' %list(result.pvalues)[i]
    f_p = '%.6f' %result.f_pvalue
    print('指数',index_file[:4],'\n系数',index_save_dct,'\nP值',index_p_save_dct,'\nF统计量P值',f_p)

    # 用arch
    # reg = arch_model(y, x=X, mean='ARX', lags=0, vol='Garch', p=1, o=0, q=1, power=1, dist='ged', hold_back=None)
    # result = reg.fit()
    # result.summary()

指数 SZCZ 
系数 {1: '-0.000783', 2: '0.000949', 3: '0.000169', 4: '-0.002108', 5: '0.000301'} 
P值 {1: '0.352362', 2: '0.250988', 3: '0.837120', 4: '0.010647', 5: '0.715876'} 
F统计量P值 0.082460
指数 SZZS 
系数 {1: '-0.000226', 2: '0.000664', 3: '0.000213', 4: '-0.001696', 5: '0.000688'} 
P值 {1: '0.750851', 2: '0.341403', 3: '0.758506', 4: '0.014909', 5: '0.324896'} 
F统计量P值 0.093651
指数 CYBZ 
系数 {1: '0.000321', 2: '0.001084', 3: '0.000801', 4: '-0.002286', 5: '0.000290'} 
P值 {1: '0.755526', 2: '0.283907', 3: '0.425059', 4: '0.023571', 5: '0.774437'} 
F统计量P值 0.131635


In [291]:
# 指数角度检验 - 分时段

for time in range(5):
    
    if time ==0:
        start = '2011-01-04'
        end = '2014-12-31'
    if time ==1:
        start = '2012-01-01'
        end = '2015-12-31'
    if time ==2:
        start = '2013-01-01'
        end = '2016-12-31'
    if time ==3:
        start = '2014-01-01'
        end = '2017-12-31'
    if time ==4:
        start = '2015-01-01'
        end = '2018-12-31'
    
    index_save_dct = {1:0,2:0,3:0,4:0,5:0}
    index_p_save_dct = {1:0,2:0,3:0,4:0,5:0}
    for index_file in ['SZCZ_lnr_rv_w_m_ntd_080101-','SZZS_lnr_rv_w_m_ntd_080101-','CYBZ_lnr_rv_w_m_ntd_100601-']:
        index_df = pd.read_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/'+index_file+'190131adj.csv',index_col=0)

        index_df = index_df[index_df.index>start]
        index_df = index_df[index_df.index<end]
        X = dummy_df[dummy_df.index>start]
        X = X[X.index<end]
        y = index_df['lnR']

        # 用OLS
    #     X = sm.add_constant(X)
        est = sm.OLS(y,X)
        result = est.fit()
        for i in range(5):
            index_save_dct[i+1] = '%.6f' %list(result.params)[i]
        for i in range(5):
            index_p_save_dct[i+1] = '%.6f' %list(result.pvalues)[i]
        f_p = '%.6f' %result.f_pvalue
        print('时段',time+1,'指数',index_file[:4],'\n系数',index_save_dct,'\nP值',index_p_save_dct,'\nF统计量P值',f_p,'\n')

时段 1 指数 SZCZ 
系数 {1: '-0.001443', 2: '0.000094', 3: '0.001110', 4: '-0.002529', 5: '0.001846'} 
P值 {1: '0.151657', 2: '0.924683', 3: '0.261218', 4: '0.010380', 5: '0.061089'} 
F统计量P值 0.010309 

时段 1 指数 SZZS 
系数 {1: '-0.000498', 2: '0.000090', 3: '0.000945', 4: '-0.001863', 5: '0.001844'} 
P值 {1: '0.542501', 2: '0.911670', 3: '0.239528', 4: '0.020188', 5: '0.021482'} 
F统计量P值 0.015052 

时段 1 指数 CYBZ 
系数 {1: '0.000895', 2: '-0.000866', 3: '0.002342', 4: '-0.002842', 5: '0.001702'} 
P值 {1: '0.490130', 2: '0.498358', 3: '0.065961', 4: '0.025370', 5: '0.180218'} 
F统计量P值 0.027639 

时段 2 指数 SZCZ 
系数 {1: '-0.000044', 2: '0.000742', 3: '0.001152', 4: '-0.001236', 5: '0.001275'} 
P值 {1: '0.972932', 2: '0.565841', 3: '0.371500', 4: '0.340174', 5: '0.326602'} 
F统计量P值 0.631493 

时段 2 指数 SZZS 
系数 {1: '0.000688', 2: '0.000340', 3: '0.000895', 4: '-0.000851', 5: '0.001436'} 
P值 {1: '0.545549', 2: '0.762728', 3: '0.425459', 4: '0.451093', 5: '0.204566'} 
F统计量P值 0.682839 

时段 2 指数 CYBZ 
系数 {1: '0.002363'

In [296]:
# 个股角度检验

code_lst = os.listdir('E:/Stock_Data/stock_return_data')
X = dummy_df.iloc[:1943,:] 
result_save_dct = {1:0,2:0,3:0,4:0,5:0}
cnt = 0
tol_sig = 0
positive = {1:0,2:0,3:0,4:0,5:0}
pvalue = {1:0,2:0,3:0,4:0,5:0}
pvalue_pos = {1:0,2:0,3:0,4:0,5:0}
f_pvalue = 0
for code_csv in code_lst:
    y_temp = pd.read_csv('E:/Stock_Data/stock_return_data/'+code_csv,index_col = 0)
    if len(y_temp)<1943:
        continue
    y = y_temp.iloc[:1943,:]
    
    # 用OLS
#     X = sm.add_constant(X)
    est = sm.OLS(y,X)
    result = est.fit()
    
    # 统计专用
    
    # 显著结果统计(有显著性水平要求0.1)
    for i in range(5):
        if list(result.pvalues)[i]<0.05:
            pvalue[i+1] += 1          
    # 显著结果中正负统计(有显著性水平要求0.1)
    for i in range(5):
        if list(result.pvalues)[i]<0.05 and list(result.params)[i]>0:
            pvalue_pos[i+1] += 1
    
    # 最显著结果统计(无显著性水平要求)
    result_save_dct[np.argmin(list(result.pvalues))+1]+=1    
    
    # 正负统计-回归、其实就是样本均值
    for i in range(5):
        if list(result.params)[i]>0:
            positive[i+1] += 1
            
    # F统计量显著统计
    if result.f_pvalue < 0.1:
        f_pvalue += 1
                
#     # 频率统计，周四为负
#     y_Thu = y[X[4]==1]['return']
#     y_Thu_std = y_Thu.std()
#     Thu_neg += sum(y_Thu.apply(lambda x: 1 if x<-3*y_Thu_std else 0))
#     Thu_pos += sum(y_Thu.apply(lambda x: 1 if x> 3*y_Thu_std else 0))
#     # 频率统计，周一为正
#     y_Mon = y[(X[2]==0) & (X[3]==0) & (X[4]==0) & (X[5]==0)]['return']
#     y_Mon_std = y_Mon.std()
#     Mon_pos += sum(y_Mon.apply(lambda x: 1 if x> 3*y_Mon_std else 0))
#     Mon_neg += sum(y_Mon.apply(lambda x: 1 if x<-3*y_Mon_std else 0))
    # 频率统计，平均数正负
    
    # 频率统计，周四小于周一
#     y_Mon = y[(X[2]==0) & (X[3]==0) & (X[4]==0) & (X[5]==0) ]['return']
#     y_Thu = y[X[4]==1]['return']
#     y_delta = y_Thu - y_Mon
#     Thu_lower_Mon += sum(y_delta.apply(lambda x: 1 if x<0 else 0))  
    
    # 用arch
#     reg = arch_model(y, x=X, mean='ARX', lags=0, vol='Garch', p=1, o=0, q=1, power=1, dist='ged', hold_back=None)
#     result = reg.fit()
#     result_save_dct[np.argmin(list(result.pvalues[:5]))+1]+=1
        
    cnt += 1
    tol_sig += list(result.pvalues)[4]

In [297]:
print('显著结果统计',pvalue,'\n显著结果中正负统计',pvalue_pos,'\n最显著结果统计',result_save_dct,'\n回归结果正负统计',positive,'\nF统计量显著数',f_pvalue,'\n总数',cnt)

显著结果统计 {1: 40, 2: 58, 3: 21, 4: 360, 5: 29} 
显著结果中正负统计 {1: 12, 2: 58, 3: 21, 4: 0, 5: 28} 
最显著结果统计 {1: 103, 2: 111, 3: 60, 4: 714, 5: 81} 
回归结果正负统计 {1: 438, 2: 886, 3: 834, 4: 12, 5: 620} 
F统计量显著数 350 
总数 1069


In [256]:
result.f_pvalue

0.23459151003391704